<h1><font color=blue>Week 3 Assignment of Applied Data Science Capstone</font></h1>

<h3><font color=black>Task 3: Add the latitude and longitude coordinates to the dataframe</font></h3>

#### importing dependencies

In [67]:
import pandas as pd
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

#### importing csv created from DF in task 2

In [150]:
postalcode_df = pd.read_csv("week3_assignment2.csv")
postalcode_df.dropna()

Borough                                      Neighbourhood  \
0         Scarborough                                      Rouge,Malvern   
1         Scarborough               Highland Creek,Rouge Hill,Port Union   
2         Scarborough                    Guildwood,Morningside,West Hill   
3         Scarborough                                             Woburn   
4         Scarborough                                          Cedarbrae   
5         Scarborough                                Scarborough Village   
6         Scarborough          East Birchmount Park,Ionview,Kennedy Park   
7         Scarborough                      Clairlea,Golden Mile,Oakridge   
8         Scarborough      Cliffcrest,Cliffside,Scarborough Village West   
9         Scarborough                         Birch Cliff,Cliffside West   
10        Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...   
11        Scarborough                                   Maryvale,Wexford   
12        Scarborough                                          Agincourt   
13        Scarborough              Clarks Corners,Sullivan,Tam O'Shanter   
14        Scarborough  Agincourt North,L'Amoreaux East,Milliken,Steel...   
15        Scarborough                                    L'Amoreaux West   
16        Scarborough                                        Upper Rouge   
17         North York                                  Hillcrest Village   
18         North York                         Fairview,Henry Farm,Oriole   
19         North York                                    Bayview Village   
20         North York                            Silver Hills,York Mills   
21         North York                             Newtonbrook,Willowdale   
22         North York                                   Willowdale South   
23         North York                                    York Mills West   
24         North York                                    Willowdale West   
25         North York                                          Parkwoods   
26         North York                                    Don Mills North   
27         North York                    Flemingdon Park,Don Mills South   
28         North York      Bathurst Manor,Downsview North,Wilson Heights   
29         North York                     Northwood Park,York University   
..                ...                                                ...   
73               York                                 Humewood-Cedarvale   
74               York                                Caledonia-Fairbanks   
75   Downtown Toronto                                           Christie   
76       West Toronto                        Dovercourt Village,Dufferin   
77       West Toronto                            Little Portugal,Trinity   
78       West Toronto         Brockton,Exhibition Place,Parkdale Village   
79         North York                   Downsview,North Park,Upwood Park   
80               York        Del Ray,Keelesdale,Mount Dennis,Silverthorn   
81               York                       The Junction North,Runnymede   
82       West Toronto                       High Park,The Junction South   
83       West Toronto                              Parkdale,Roncesvalles   
84       West Toronto                                  Runnymede,Swansea   
85       Queen's Park                                       Queen's Park   
86        Mississauga              Canada Post Gateway Processing Centre   
87       East Toronto  Business Reply Mail Processing Centre 969 Eastern   
88          Etobicoke         Humber Bay Shores,Mimico South,New Toronto   
89          Etobicoke                              Alderwood,Long Branch   
90          Etobicoke        The Kingsway,Montgomery Road,Old Mill North   
91          Etobicoke  Humber Bay,King's Mill Park,Kingsway Park Sout...   
92          Etobicoke  Kingsway Park South West,Mimico NW,The Queensw...   
93          Etobicoke                                   Isling

#### let's visualize Toronto and  the neighborhoods in it.

In [128]:
#Generating initial map
latitude=43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude, longitude], tiles='Stamen Terrain', zoom_start=10)
map_toronto

# adding markers to map of tornoto Canada
for lat, lng, borough, neighborhood in zip(postalcode_df['Latitude'], \
                                           postalcode_df['Longitude'], \
                                           postalcode_df['Borough'], \
                                           postalcode_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [4]:
#importing client ID and client secret from config file. used this method so i can use git ignore to protect api key
from config import client_id, client_secret

#### creating API variables

In [166]:

CLIENT_ID = client_id 
CLIENT_SECRET = client_secret 
VERSION = '20180605' 

#### importing required libraries


In [8]:
from sklearn.cluster import KMeans
import numpy as np
import requests

#### Let's create a function to explore all the neighborhoods in Toronto

In [9]:
#the function below pulls the top 100 venues within a 500m radius of toronto neighborhoods
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [167]:
#calling the function to get nearby venues
toronto_venues = getNearbyVenues(names=postalcode_df['Neighbourhood'],
                                   latitudes=postalcode_df['Latitude'],
                                   longitudes=postalcode_df['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

#### Let's check the size of the resulting dataframe

In [11]:
#viewing the df 
print(toronto_venues.shape)
toronto_venues.head()

(2245, 7)


Neighborhood  Neighborhood Latitude  \
0                         Rouge,Malvern              43.806686   
1  Highland Creek,Rouge Hill,Port Union              43.784535   
2  Highland Creek,Rouge Hill,Port Union              43.784535   
3       Guildwood,Morningside,West Hill              43.763573   
4       Guildwood,Morningside,West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.160497            Royal Canadian Legion       43.782533   
2              -79.160497   Scarborough Historical Society       43.788755   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.162438        History Museum  
3       -79.189914           Pizza Place  
4       -79.191537     Electronics Store

 #### Let's find out how many unique categories can be curated from all the returned venues

In [12]:
unique_categories = len(toronto_venues['Venue Category'].unique())
print(f'There are {unique_categories} uniques categories')

There are 267 uniques categories


#### prepping df for the model by converting df to int

In [130]:
# one hot encoding to convert strings to numerical values
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0               0                  0                  0        0   
1               0                  0                  0        0   
2               0                  0                  0        0   
3               0                  0                  0        0   
4               0                  0                  0        0   
5               0                  0                  0        0   
6               0                  0                  0        0   
7               0                  0                  0        0   
8               0                  0                  0        0   
9               0                  0                  0        0   
10              0                  0                  0        0   
11              0                  0                  0        0   
12              0                  0                  0        0   
13              0                  0                  0        0   
14              0                  0                  0        0   
15              0                  0                  0        0   
16              0                  0                  0        0   
17              0                  0                  0        0   
18              0                  0                  0        0   
19              0                  0                  0        0   
20              0                  0                  0        0   
21              0                  0                  0        0   
22              0                  0                  0        0   
23              0                  0                  0        0   
24              0                  0                  0        0   
25              0                  0                  0        0   
26              0                  0                  0        0   
27              0                  0                  0        0   
28              0                  0                  0        0   
29              0                  0                  0        0   
...           ...                ...                ...      ...   
2215            0                  0                  0        0   
2216            0                  0                  0        0   
2217            0                  0                  0        0   
2218            0                  0                  0        0   
2219            0                  0                  0        0   
2220            0                  0                  0        0   
2221            0                  0                  0        0   
2222            0                  0                  0        0   
2223            0                  0                  0        0   
2224            0                  0                  0        0   
2225            0                  0                  0        0   
2226            0                  0                  0        0   
2227            0                  0                  0        0   
2228            0                  0                  0        0   
2229            0                  0                  0        0   
2230            0                  0                  0        0   
2231            0                  0                  0        0   
2232            0                  0                  0        0   
2233            0                  0                  0        0   
2234            0                  0                  0        0   
2235            0                  0                  0        0   
2236            0                  0                  0        0   
2237            0                  0                  0        0   
2238            0                  0                  0        0   
2239            0                  0                  0        0   
2240            0                  0                  0        0   
2241            0                  0                  0       

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [132]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
# toronto_venues.head()

Neighborhood  Yoga Studio  \
0                              Adelaide,King,Richmond     0.000000   
1                                           Agincourt     0.000000   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...     0.000000   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...     0.000000   
4                               Alderwood,Long Branch     0.000000   
5       Bathurst Manor,Downsview North,Wilson Heights     0.000000   
6                                     Bayview Village     0.000000   
7                    Bedford Park,Lawrence Manor East     0.000000   
8                                         Berczy Park     0.000000   
9                          Birch Cliff,Cliffside West     0.000000   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...     0.000000   
11         Brockton,Exhibition Place,Parkdale Village     0.000000   
12  Business Reply Mail Processing Centre 969 Eastern     0.058824   
13                         CFB Toronto,Downsview East     0.000000   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...     0.000000   
15                         Cabbagetown,St. James Town     0.000000   
16                                Caledonia-Fairbanks     0.000000   
17              Canada Post Gateway Processing Centre     0.000000   
18                                          Cedarbrae     0.000000   
19                                 Central Bay Street     0.012195   
20            Chinatown,Grange Park,Kensington Market     0.000000   
21                                           Christie     0.000000   
22                               Church and Wellesley     0.010989   
23                      Clairlea,Golden Mile,Oakridge     0.000000   
24              Clarks Corners,Sullivan,Tam O'Shanter     0.000000   
25      Cliffcrest,Cliffside,Scarborough Village West     0.000000   
26                      Commerce Court,Victoria Hotel     0.000000   
27                                         Davisville     0.000000   
28                                   Davisville North     0.000000   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...     0.000000   
..                                                ...          ...   
69                                          Northwest     0.000000   
70                     Northwood Park,York University     0.000000   
71                              Parkdale,Roncesvalles     0.000000   
72                                          Parkwoods     0.000000   
73                                       Queen's Park     0.025641   
74                                           Rosedale     0.000000   
75                                           Roselawn     0.000000   
76                                      Rouge,Malvern     0.000000   
77                                  Runnymede,Swansea     0.000000   
78                            Ryerson,Garden District     0.000000   
79                                Scarborough Village     0.000000   
80                                     St. James Town     0.000000   
81                    Stn A PO Boxes 25 The Esplanade     0.000000   
82                                    Studio District     0.026316   
83                  The Annex,North Midtown,Yorkville     0.000000   
84                                        The Beaches     0.000000   
85                      The Beaches West,India Bazaar     0.000000   
86                        The Danforth West,Riverdale     0.023810   
87                       The Junction North,Runnymede     0.000000   
88        The Kingsway,Montgomery Road,Old Mill North     0.000000   
89                                   Thorncliffe Park     0.055556   
90                                   Victoria Village     0.000000   
91                                          Westmount     0.000000   
92                                             Weston     0.000000   
93                                   Willowdale South     0.000000   
94                                    Willowdale We

#### Let's confirm the new size

In [133]:
toronto_grouped.shape

(99, 267)

#### Let's print each neighborhood along with the top 5 most common venues

In [134]:
num_top_venues = 5

for hood in (toronto_grouped['Neighborhood']):
    print(f"----{hood}----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2               Bar  0.04
3  Sushi Restaurant  0.03
4             Hotel  0.03


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1               Skating Rink  0.25
2                     Lounge  0.25
3             Breakfast Spot  0.25
4                      Motel  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0          Pizza Place  0.14
1           Beer Store  0.14
2  Fried Chicken Joint  0.14
3        Grocery Store  0.14
4       Sandwich Place  0.14


----Alderwood,Long B

4               Motel  0.00


----Emery,Humberlea----
               venue  freq
0     Baseball Field   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.12
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3         Women's Store  0.05
4           Bus Station  0.03


----First Canadian Place,Underground city----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.07
2   Restaurant  0.04
3        Hotel  0.04
4   Steakhouse  0.04


----Flemingdon Park,Don Mills South----
          venue  freq
0    Beer Store  0.10
1   Coffee Shop  0.10
2           Gym  0.10
3  Concert Hall  0.05
4     Bike Shop  0.05


----Forest Hill North,Forest Hill West----
              venue  freq
0             Trail   0.2
1              Park   0.2
2      Home Service   0.2
3  Sushi Restaurant   0.2
4     Jewelry Store   0.2


----Glencairn---

4  American Restaurant  0.05


----The Annex,North Midtown,Yorkville----
              venue  freq
0    Sandwich Place  0.13
1              Café  0.13
2       Coffee Shop  0.09
3    History Museum  0.04
4  Asian Restaurant  0.04


----The Beaches----
                       venue  freq
0          Health Food Store   0.2
1                        Pub   0.2
2       Other Great Outdoors   0.2
3                      Trail   0.2
4  Middle Eastern Restaurant   0.0


----The Beaches West,India Bazaar----
                venue  freq
0                Park  0.09
1      Sandwich Place  0.09
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.05


----The Danforth West,Riverdale----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.07
4  Furniture / Home Store  0.05


----The Junction North,Runnymede----
                  venue  freq
0     Convenience Store  0.25
1 

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood   1st Most Common Venue  \
0                              Adelaide,King,Richmond             Coffee Shop   
1                                           Agincourt            Skating Rink   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...              Playground   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...    Fast Food Restaurant   
4                               Alderwood,Long Branch             Pizza Place   
5       Bathurst Manor,Downsview North,Wilson Heights             Coffee Shop   
6                                     Bayview Village                    Café   
7                    Bedford Park,Lawrence Manor East      Italian Restaurant   
8                                         Berczy Park             Coffee Shop   
9                          Birch Cliff,Cliffside West         College Stadium   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...                Pharmacy   
11         Brockton,Exhibition Place,Parkdale Village                    Café   
12  Business Reply Mail Processing Centre 969 Eastern      Light Rail Station   
13                         CFB Toronto,Downsview East                 Airport   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...         Airport Service   
15                         Cabbagetown,St. James Town             Coffee Shop   
16                                Caledonia-Fairbanks                    Park   
17              Canada Post Gateway Processing Centre             Coffee Shop   
18                                          Cedarbrae        Hakka Restaurant   
19                                 Central Bay Street             Coffee Shop   
20            Chinatown,Grange Park,Kensington Market                    Café   
21                                           Christie                    Café   
22                               Church and Wellesley             Coffee Shop   
23                      Clairlea,Golden Mile,Oakridge                  Bakery   
24              Clarks Corners,Sullivan,Tam O'Shanter             Pizza Place   
25      Cliffcrest,Cliffside,Scarborough Village West     American Restaurant   
26                      Commerce Court,Victoria Hotel             Coffee Shop   
27                                         Davisville             Pizza Place   
28                                   Davisville North    Gym / Fitness Center   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...             Coffee Shop   
..                                                ...                     ...   
69                                          Northwest                     Bar   
70                     Northwood Park,York University  Furniture / Home Store   
71                              Parkdale,Roncesvalles             Coffee Shop   
72                                          Parkwoods       Food & Drink Shop   
73                                       Queen's Park             Coffee Shop   
74                                           Rosedale                    Park   
75                                           Roselawn                  Garden   
76                                      Rouge,Malvern    Fast Food Restaurant   
77                                  Runnymede,Swansea             Coffee Shop   
78                            Ryerson,Garden District          Clothing Store   
79                                Scarborough Village              Playground   
80                                     St. James Town             Coffee Shop   
81                    Stn A PO Boxes 25 The Esplanade             Coffee Shop   
82                                    Studio District                    Café   
83                  The Annex,North Midtown,Yorkville          Sandwich Place   
84                                        The Beaches                     Pub   
85                      The Beaches West,India Bazaar                    Park   
86                        The Danforth West,Riverdale        Greek Resta

In [138]:
postalcode_df.head(2)

Borough                         Neighbourhood   Latitude  Longitude
0  Scarborough                         Rouge,Malvern  43.806686 -79.194353
1  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535 -79.160497

Run k-means to cluster the neighborhood into 5 clusters.

In [139]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [140]:
neighborhoods_venues_sorted

Neighborhood   1st Most Common Venue  \
0                              Adelaide,King,Richmond             Coffee Shop   
1                                           Agincourt            Skating Rink   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...              Playground   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...    Fast Food Restaurant   
4                               Alderwood,Long Branch             Pizza Place   
5       Bathurst Manor,Downsview North,Wilson Heights             Coffee Shop   
6                                     Bayview Village                    Café   
7                    Bedford Park,Lawrence Manor East      Italian Restaurant   
8                                         Berczy Park             Coffee Shop   
9                          Birch Cliff,Cliffside West         College Stadium   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...                Pharmacy   
11         Brockton,Exhibition Place,Parkdale Village                    Café   
12  Business Reply Mail Processing Centre 969 Eastern      Light Rail Station   
13                         CFB Toronto,Downsview East                 Airport   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...         Airport Service   
15                         Cabbagetown,St. James Town             Coffee Shop   
16                                Caledonia-Fairbanks                    Park   
17              Canada Post Gateway Processing Centre             Coffee Shop   
18                                          Cedarbrae        Hakka Restaurant   
19                                 Central Bay Street             Coffee Shop   
20            Chinatown,Grange Park,Kensington Market                    Café   
21                                           Christie                    Café   
22                               Church and Wellesley             Coffee Shop   
23                      Clairlea,Golden Mile,Oakridge                  Bakery   
24              Clarks Corners,Sullivan,Tam O'Shanter             Pizza Place   
25      Cliffcrest,Cliffside,Scarborough Village West     American Restaurant   
26                      Commerce Court,Victoria Hotel             Coffee Shop   
27                                         Davisville             Pizza Place   
28                                   Davisville North    Gym / Fitness Center   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...             Coffee Shop   
..                                                ...                     ...   
69                                          Northwest                     Bar   
70                     Northwood Park,York University  Furniture / Home Store   
71                              Parkdale,Roncesvalles             Coffee Shop   
72                                          Parkwoods       Food & Drink Shop   
73                                       Queen's Park             Coffee Shop   
74                                           Rosedale                    Park   
75                                           Roselawn                  Garden   
76                                      Rouge,Malvern    Fast Food Restaurant   
77                                  Runnymede,Swansea             Coffee Shop   
78                            Ryerson,Garden District          Clothing Store   
79                                Scarborough Village              Playground   
80                                     St. James Town             Coffee Shop   
81                    Stn A PO Boxes 25 The Esplanade             Coffee Shop   
82                                    Studio District                    Café   
83                  The Annex,North Midtown,Yorkville          Sandwich Place   
84                                        The Beaches                     Pub   
85                      The Beaches West,India Bazaar                    Park   
86                        The Danforth West,Riverdale        Greek Resta

Adding clustering labels and  merging dataframes

In [151]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = postalcode_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged = toronto_merged.dropna() 

toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

toronto_merged # check the last columns!

Borough                                      Neighbourhood  \
0         Scarborough                                      Rouge,Malvern   
1         Scarborough               Highland Creek,Rouge Hill,Port Union   
2         Scarborough                    Guildwood,Morningside,West Hill   
3         Scarborough                                             Woburn   
4         Scarborough                                          Cedarbrae   
5         Scarborough                                Scarborough Village   
6         Scarborough          East Birchmount Park,Ionview,Kennedy Park   
7         Scarborough                      Clairlea,Golden Mile,Oakridge   
8         Scarborough      Cliffcrest,Cliffside,Scarborough Village West   
9         Scarborough                         Birch Cliff,Cliffside West   
10        Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...   
11        Scarborough                                   Maryvale,Wexford   
12        Scarborough                                          Agincourt   
13        Scarborough              Clarks Corners,Sullivan,Tam O'Shanter   
14        Scarborough  Agincourt North,L'Amoreaux East,Milliken,Steel...   
15        Scarborough                                    L'Amoreaux West   
17         North York                                  Hillcrest Village   
18         North York                         Fairview,Henry Farm,Oriole   
19         North York                                    Bayview Village   
21         North York                             Newtonbrook,Willowdale   
22         North York                                   Willowdale South   
23         North York                                    York Mills West   
24         North York                                    Willowdale West   
25         North York                                          Parkwoods   
26         North York                                    Don Mills North   
27         North York                    Flemingdon Park,Don Mills South   
28         North York      Bathurst Manor,Downsview North,Wilson Heights   
29         North York                     Northwood Park,York University   
30         North York                         CFB Toronto,Downsview East   
31         North York                                     Downsview West   
..                ...                                                ...   
71         North York                    Lawrence Heights,Lawrence Manor   
72         North York                                          Glencairn   
73               York                                 Humewood-Cedarvale   
74               York                                Caledonia-Fairbanks   
75   Downtown Toronto                                           Christie   
76       West Toronto                        Dovercourt Village,Dufferin   
77       West Toronto                            Little Portugal,Trinity   
78       West Toronto         Brockton,Exhibition Place,Parkdale Village   
79         North York                   Downsview,North Park,Upwood Park   
80               York        Del Ray,Keelesdale,Mount Dennis,Silverthorn   
81               York                       The Junction North,Runnymede   
82       West Toronto                       High Park,The Junction South   
83       West Toronto                              Parkdale,Roncesvalles   
84       West Toronto                                  Runnymede,Swansea   
85       Queen's Park                                       Queen's Park   
86        Mississauga              Canada Post Gateway Processing Centre   
87       East Toronto  Business Reply Mail Processing Centre 969 Eastern   
88          Etobicoke         Humber Bay Shores,Mimico South,New Toronto   
89          Etobicoke                              Alderwood,Long Branch   
90          Etobicoke        The Kingsway,Montgomery Road,Old Mill North   
91          Etobicoke  Humber Bay,King's Mill Park,Kingsway Pa

Finally, let's visualize the resulting clusters

In [157]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [158]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood  \
0                                        Rouge,Malvern   
1                 Highland Creek,Rouge Hill,Port Union   
2                      Guildwood,Morningside,West Hill   
3                                               Woburn   
4                                            Cedarbrae   
6            East Birchmount Park,Ionview,Kennedy Park   
7                        Clairlea,Golden Mile,Oakridge   
8        Cliffcrest,Cliffside,Scarborough Village West   
9                           Birch Cliff,Cliffside West   
10   Dorset Park,Scarborough Town Centre,Wexford He...   
11                                    Maryvale,Wexford   
12                                           Agincourt   
13               Clarks Corners,Sullivan,Tam O'Shanter   
15                                     L'Amoreaux West   
17                                   Hillcrest Village   
18                          Fairview,Henry Farm,Oriole   
19                                     Bayview Village   
22                                    Willowdale South   
24                                     Willowdale West   
26                                     Don Mills North   
27                     Flemingdon Park,Don Mills South   
28       Bathurst Manor,Downsview North,Wilson Heights   
29                      Northwood Park,York University   
31                                      Downsview West   
32                                   Downsview Central   
33                                 Downsview Northwest   
34                                    Victoria Village   
35                      Woodbine Gardens,Parkview Hill   
36                                    Woodbine Heights   
37                                         The Beaches   
..                                                 ...   
66                       Harbord,University of Toronto   
67             Chinatown,Grange Park,Kensington Market   
68   CN Tower,Bathurst Quay,Island airport,Harbourf...   
69                     Stn A PO Boxes 25 The Esplanade   
70               First Canadian Place,Underground city   
71                     Lawrence Heights,Lawrence Manor   
72                                           Glencairn   
73                                  Humewood-Cedarvale   
75                                            Christie   
76                         Dovercourt Village,Dufferin   
77                             Little Portugal,Trinity   
78          Brockton,Exhibition Place,Parkdale Village   
79                    Downsview,North Park,Upwood Park   
80         Del Ray,Keelesdale,Mount Dennis,Silverthorn   
81                        The Junction North,Runnymede   
82                        High Park,The Junction South   
83                               Parkdale,Roncesvalles   
84                                   Runnymede,Swansea   
85                                        Queen's Park   
86               Canada Post Gateway Processing Centre   
87   Business Reply Mail Processing Centre 969 Eastern   
88          Humber Bay Shores,Mimico South,New Toronto   
89                               Alderwood,Long Branch   
92   Kingsway Park South West,Mimico NW,The Queensw...   
95   Bloordale Gardens,Eringate,Markland Wood,Old B...   
96                                       Humber Summit   
99                                           Westmount   
100  Kingsview Village,Martin Grove Gardens,Richvie...   
101  Albion Gardens,Beaumond Heights,Humbergate,Jam...   
102                                          Northwest   

      1st Most Common Venue       2nd Most Common Venue  \
0      Fast Food Restaurant               Women's Store   
1            History Museum                         Bar   
2               Pizza Place              Breakfast Spot   
3               Coffee Shop           Indian Restaurant   
4          Hakka Restaurant                      Bakery   
6            Discount Store                 Bus Station   
7                    Bakery    

Finally, let's visualize the resulting clusters

In [159]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
23                              York Mills West                  Park   
25                                    Parkwoods     Food & Drink Shop   
30                   CFB Toronto,Downsview East               Airport   
40                                 East Toronto                  Park   
44                                Lawrence Park              Bus Line   
50                                     Rosedale                  Park   
74                          Caledonia-Fairbanks                  Park   
90  The Kingsway,Montgomery Road,Old Mill North                  Park   
98                                       Weston                  Park   

   2nd Most Common Venue        3rd Most Common Venue  \
23                  Bank            Convenience Store   
25                  Park  Eastern European Restaurant   
30                  Park                Women's Store   
40     Convenience Store                Women's Store   
44                  Park                  Swim School   
50                 Trail                   Playground   
74         Women's Store         Fast Food Restaurant   
90                  Pool                        River   
98         Women's Store  Eastern European Restaurant   

          4th Most Common Venue        5th Most Common Venue  \
23                Women's Store  Eastern European Restaurant   
25               Discount Store                      Dog Run   
30  Eastern European Restaurant               Discount Store   
40  Eastern European Restaurant                      Dog Run   
44                Women's Store          Dumpling Restaurant   
50                Women's Store                        Diner   
74                       Market           Falafel Restaurant   
90                Women's Store                    Drugstore   
98               Discount Store                      Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
23               Dog Run      Doner Restaurant            Donut Shop   
25      Doner Restaurant            Donut Shop             Drugstore   
30               Dog Run      Doner Restaurant            Donut Shop   
40      Doner Restaurant            Donut Shop             Drugstore   
44               Dog Run      Doner Restaurant            Donut Shop   
50        Discount Store               Dog Run      Doner Restaurant   
74           Event Space  Ethiopian Restaurant   Empanada Restaurant   
90                 Diner        Discount Store               Dog Run   
98      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue       10th Most Common Venue  
23             Drugstore          Dumpling Restaurant  
25   Dumpling Restaurant                Women's Store  
30             Drugstore          Dumpling Restaurant  
40   Dumpling Restaurant          Empanada Restaurant  
44             Drugstore  Eastern European Restaurant  
50            Donut Shop                    Drugstore  
74                 Diner  Eastern European Restaurant  
90      Doner Restaurant                   Donut Shop  
98   Dumpling Restaurant            Electronics Store

In [160]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
21  Newtonbrook,Willowdale                   Gym         Women's Store   

          3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
21  Eastern European Restaurant        Discount Store               Dog Run   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
21      Doner Restaurant            Donut Shop             Drugstore   

   9th Most Common Venue 10th Most Common Venue  
21   Dumpling Restaurant      Electronics Store

In [161]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
91  Humber Bay,King's Mill Park,Kingsway Park Sout...        Baseball Field   
97                                    Emery,Humberlea        Baseball Field   

   2nd Most Common Venue        3rd Most Common Venue 4th Most Common Venue  \
91         Women's Store  Eastern European Restaurant               Dog Run   
97         Women's Store  Eastern European Restaurant               Dog Run   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
91      Doner Restaurant            Donut Shop             Drugstore   
97      Doner Restaurant            Donut Shop             Drugstore   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
91   Dumpling Restaurant     Electronics Store    Filipino Restaurant  
97   Dumpling Restaurant     Electronics Store    Filipino Restaurant

In [162]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
5                                 Scarborough Village            Playground   
14  Agincourt North,L'Amoreaux East,Milliken,Steel...            Playground   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5     Dim Sum Restaurant        Farmers Market    Falafel Restaurant   
14                  Park   Dumpling Restaurant                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5            Event Space  Ethiopian Restaurant   Empanada Restaurant   
14        Discount Store               Dog Run      Doner Restaurant   

   8th Most Common Venue        9th Most Common Venue  \
5      Electronics Store  Eastern European Restaurant   
14            Donut Shop                    Drugstore   

         10th Most Common Venue  
5           Dumpling Restaurant  
14  Eastern European Restaurant